# Iterative Prisoner's Dilemma


### Description

The [Prisoner's Dilemma](https://en.wikipedia.org/wiki/Prisoner%27s_dilemma) (PD) is a classical game analyzed in game theory, which is widely used to (attempt to) model social/economical interaction. It's a "dilemma" as, if exploited to explain the emergence of altruism in human or in general animal society, it fails badly at a first glance.

The classical situation-representation of the PD is that of two prisoners whose conviction depends on their mutual cooperation. It is easier understood though if illustrated in terms of a trade-off game (closed bag exachange):

*Two people meet and exchange closed bags, with the understanding that one of them contains money, and the other contains a purchase. Either player can choose to honor the deal by putting into his or her bag what he or she agreed, or he or she can defect by handing over an empty bag.*

It is obvious that for both players the winning strategy is to NOT cooperate.

Things changes when the interaction between the two individuals is iterated, in that case a more altruist attitude (strategy) is expected to emerge. The goal of this project is to test this hypothesis.

Mathematically the PD can be expressed with very basic linear algebra. The key component is the **Payoff matrix** $M$, which quantify the reward each player gets depending on whether she cooperated or not (defect):

$$
M = 
\begin{pmatrix} 
R & S \\
T & P 
\end{pmatrix}
$$

with $T,R,S,P$ integers that satisfy the following conditions:

$$
T>R>P>S; \quad 2R > T+S
$$

for example $T=3$, $R=2$, $P=1$ and $S=0$, or  $T=5$, $R=3$, $P=2$, $S=0$. Each player choice (move) can be represented by one of the two axis in ${\rm I\!R}^2$, i.e. $u_C=\begin{pmatrix} 1 \\ 0 \end{pmatrix}$ or $u_D=\begin{pmatrix} 0 \\ 1 \end{pmatrix}$, where the first coordinate stands for *Cooperate* and the second for *Defect*. Being $u_1$ and $u_2$ their rewards $r_1$ and $r_2$ can be computed then as:

$$
r_1 = u_1^T M u_2
\quad
\quad
r_2 = u_2^T M u_1
$$

In an Iterative Prisoner's Dilemma (IPD), two players play prisoner's dilemma more than once in succession and they remember previous actions of their opponent and change their strategy accordingly. The winning strategy is the one which yields to a larger reward at the end of the IPD.

The strategy can be represented as a function which outputs either $u_C$ or $u_D$. Such function can depend on the opponent's history of moves, her on history of moves, on the number of moves played till that moment and so on, but it can only be based on a probability density function. Possible strategies are:

* **Nice guy**: always cooperate (the function's output is always $u_D$)
* **Bad guy**: always defect 
* **Mainly nice**: randomly defect $k\%$ of the times and cooperate $100-k\%$, $k<50$
* **Mainly bad**: randomly defect $k\%$ of the times and cooperate $100-k\%$, $k>50$
* **tit-for-tat**: start by cooperating, then repeat what the opponent has done in the previous move 

Many more and much more complex strategies can be implemented. The strategy can even change during the IPD.


### Assignments

* Implement a simple IPD between two players implementing two given strategies. Study the evolution along the tournament confronting different strategies; study the overall outcome in the different configurations. 
* Implement a multiple players IPD (MPIPD) where several strategies play against each other in a roud-robin scheme
* Iterate what done in the previous task (repeated MPIPD, rMPIPD)  by increasing the population implementing a given strategy depending on the results that strategy achieved in the previous iteration
* (*difficult*) Implement a rMPIPD where strategies are allowed to mutate. The goal is to simulate the effect of genetic mutations and the effect of natura selection. A parameter (gene) should encode the attidue of an individual to cooperate, such gene can mutate randomly and the corresponding phenotype should compete in the MPIPD such that the best-fitted is determined.  


In [79]:
import numpy as np
import random
import numpy.random as npr
npr.seed(143) # fix seed for reproducible result

In [39]:
k = 25
u_c = np.array([1,0])
u_d = np.array([0,1])
M = np.array([[2,0], [3,1]])

In [52]:
def s_niceguy(u_1h=None, u_2h=None):
    return u_c

def s_badguy(u_1h=None, u_2h=None):
    return u_d

def s_mnice(u_1h=None, u_2h=None):
    a = npr.rand()
    if (a > k/100): return u_c
    else: return u_d

def s_mbad(u_1h=None, u_2h=None):
    a = npr.rand()
    if (a > k/100): return u_d
    else: return u_c
    
def s_titfortat(u_1h, u_2h):
    if (len(u_1h)==0): return u_c
    else: return u_2h[-1]

In [53]:
def reward(u,v): # reward of u
    return np.dot(u.T, np.dot(M,v))

In [87]:
def match(u_1, u_2):
    r1 = reward(u_1, u_2)
    r2 = reward(u_2, u_1)
    if (r1 > r2): return [r1,u_1]
    elif (r1 == r2): return [r1, u_1]
    elif (r1 < r2): return [r2, u_2]

In [88]:
strats = [s_niceguy, s_badguy, s_mnice, s_mbad, s_titfortat]

In [90]:
N = 100 #number of matches
for s_1 in strats:
    avg = 0
    for s_2 in strats:
        u_1h = []
        u_2h = []
        avg_rew = 0
        for i in range(N):
            u_1 = s_1(u_1h, u_2h)
            u_2 = s_2(u_1h, u_2h)
            u_1h.append(u_1)
            u_2h.append(u_2)
            avg_rew += match(u_1, u_2)[0]
        avg_rew = avg_rew/N
        avg += avg_rew
        print (s_1, "VS", s_2, "score:", avg_rew)
    print("the avg over all strategies of", s_1, "is :", avg/(len(strats)) )

<function s_niceguy at 0x7f5e6c08c160> VS <function s_niceguy at 0x7f5e6c08c160> score: 2.0
<function s_niceguy at 0x7f5e6c08c160> VS <function s_badguy at 0x7f5e6c08c310> score: 3.0
<function s_niceguy at 0x7f5e6c08c160> VS <function s_mnice at 0x7f5e89ad7c10> score: 2.26
<function s_niceguy at 0x7f5e6c08c160> VS <function s_mbad at 0x7f5e88144f70> score: 2.76
<function s_niceguy at 0x7f5e6c08c160> VS <function s_titfortat at 0x7f5e6c08cca0> score: 2.0
the avg over all strategies of <function s_niceguy at 0x7f5e6c08c160> is : 2.404
<function s_badguy at 0x7f5e6c08c310> VS <function s_niceguy at 0x7f5e6c08c160> score: 3.0
<function s_badguy at 0x7f5e6c08c310> VS <function s_badguy at 0x7f5e6c08c310> score: 1.0
<function s_badguy at 0x7f5e6c08c310> VS <function s_mnice at 0x7f5e89ad7c10> score: 2.46
<function s_badguy at 0x7f5e6c08c310> VS <function s_mbad at 0x7f5e88144f70> score: 1.58
<function s_badguy at 0x7f5e6c08c310> VS <function s_titfortat at 0x7f5e6c08cca0> score: 3.0
the avg 

In [84]:
n_players = 100
strats_MPIPD = random.choices(strats, k=n_players, weights=np.ones(len(strats)))#extracting strategy for each player
N = 100 #number of matches



for s_1 in strats:
    
    for s_2 in strats:
        u_1h = []
        u_2h = []
        for i in range(N):
            avg_rew = 0
            for a,s_1 in enumerate(strats_MPIPD):
                avg = 0
                for b,s_2 in enumerate(strats_MPIPD): 
                    if(a!=b):
                        u_1 = s_1(u_1h, u_2h)
                        u_2 = s_2(u_1h, u_2h)
                        u_1h.append(u_1)
                        u_2h.append(u_2)
                        avg += match(u_1, u_2)[0]
                print("rewards over all matches of", s_1, "is :", avg/(len(strats)-1) )
            avg_rew += avg_rew
        avg_rew = avg_rew/N
        avg += avg_rew
        print (s_1, "VS", s_2, "score:", avg_rew)
    print("the avg over all strategies of", s_1, "is :", avg/len(strats) )

[<function __main__.s_niceguy(u_1h=None, u_2h=None)>,
 <function __main__.s_badguy(u_1h=None, u_2h=None)>,
 <function __main__.s_niceguy(u_1h=None, u_2h=None)>,
 <function __main__.s_mbad(u_1h=None, u_2h=None)>,
 <function __main__.s_niceguy(u_1h=None, u_2h=None)>,
 <function __main__.s_badguy(u_1h=None, u_2h=None)>,
 <function __main__.s_badguy(u_1h=None, u_2h=None)>,
 <function __main__.s_mbad(u_1h=None, u_2h=None)>,
 <function __main__.s_titfortat(u_1h, u_2h)>,
 <function __main__.s_mnice(u_1h=None, u_2h=None)>,
 <function __main__.s_titfortat(u_1h, u_2h)>,
 <function __main__.s_mbad(u_1h=None, u_2h=None)>,
 <function __main__.s_mbad(u_1h=None, u_2h=None)>,
 <function __main__.s_badguy(u_1h=None, u_2h=None)>,
 <function __main__.s_niceguy(u_1h=None, u_2h=None)>,
 <function __main__.s_badguy(u_1h=None, u_2h=None)>,
 <function __main__.s_mnice(u_1h=None, u_2h=None)>,
 <function __main__.s_niceguy(u_1h=None, u_2h=None)>,
 <function __main__.s_mnice(u_1h=None, u_2h=None)>,
 <function __

In [102]:
import itertools
m=0
#for i,j in itertools.product(range(10), range(10)):
    #print(i,j)
for a,i in enumerate(strats):
    print(a,i)

0 <function s_niceguy at 0x7f5e6c08c160>
1 <function s_badguy at 0x7f5e6c08c310>
2 <function s_mnice at 0x7f5e89ad7c10>
3 <function s_mbad at 0x7f5e88144f70>
4 <function s_titfortat at 0x7f5e6c08cca0>
